In [1]:
import dask.array as da
import dask.dataframe as dd
from dask_ml.wrappers import Incremental
from sklearn.neural_network import MLPClassifier
from dask_ml.model_selection import train_test_split
from dask_ml.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
data = dd.read_csv('../data/intravehicle.csv')

In [3]:
y_data = data.pop('4')
x_data = data

In [4]:
train_x,test_x,train_y,test_y = train_test_split(x_data, y_data, test_size=0.2, random_state=0, shuffle=True)

In [5]:
train_x = train_x.reset_index(drop=True)
test_x = test_x.reset_index(drop=True) 
train_y = train_y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

In [6]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [7]:
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)

In [8]:
model = MLPClassifier(random_state=0, max_iter=100000)
inc = Incremental(model, scoring='accuracy')

In [9]:
inc.fit(train_x, train_y, classes=[0,1,2,3])

Incremental(estimator=MLPClassifier(max_iter=100000, random_state=0),
            scoring='accuracy')

In [10]:
pred_y = inc.predict(test_x)

In [11]:
pred_y = pred_y.compute()
test_y = test_y.compute()

In [ ]:
matrix = confusion_matrix(test_y, pred_y)
print(matrix)

In [ ]:
report = classification_report(test_y, pred_y, digits=5)
print(report)